In [19]:
import csv
import os.path

# Make array of just team link
teams = []

# Pull in teams file
path = r"files"

with open(os.path.join(path, 'teams.csv'), 'r') as readFile:
    reader = csv.reader(readFile)
    next(reader)  # Skip header row
    for row in reader:
        teams.append(row[1])
        
readFile.close()

In [20]:
import requests
from bs4 import BeautifulSoup

# Define base url
baseUrl = 'https://www.pro-football-reference.com'

# Iterate teams and get each team page
for team in teams:
    team_code = team.split('/')[2]
        
    # Define the output file in the teams dir
    with open(os.path.join(path, 'teams', team_code + '.csv'), 'w') as writeFile:
        writer = csv.writer(writeFile)
    
        # Get the individual team
        r = requests.get(baseUrl + team)
        soup = BeautifulSoup(r.text, 'html.parser')
    
        # Pull the headers from the table
        team_table = soup.find('table', attrs={'id':'team_index'})
        header_rows = team_table.find('thead').find_all('tr', attrs={'class': None})
            
        # Structure the header data
        header_data = []
        cols = header_rows[0].find_all('th')
        cols = [ele.text.strip().encode('utf-8') for ele in cols]
        cols.insert(1, 'Tm Link') # Add team link header column 
        header_data.append(cols)

        # Write headers to the file       
        writer.writerows(header_data)
        
        # Pull all the team data and write to file
        table_rows = team_table.find('tbody').find_all('tr', attrs={'class': None})
        
        # Team Data
        team_year_data = []

        for row in table_rows:
            cols = row.find_all('th')
            team_year = cols[0].text.strip()
            team_link = cols[0].find('a')['href']
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            cols.insert(0, team_year) # Add team name
            cols.insert(1, team_link) # Add team link
            team_year_data.append(cols)
        
        # Write data rows to the file
        writer.writerows(team_year_data)

    # Close the writer
    writeFile.close()
    
print("Team Year files complete")

Team Year files complete
